In [1]:
import theano.tensor as T
from theano import function

Couldn't import dot_parser, loading of dot files will not be possible.


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [13]:
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = function([x, y], z)

print f(2, 3)
print f(16.3, 12.1)
print type(x)
print x.type
print T.dscalar
print x.type is T.dscalar

5.0
28.4
<class 'theano.tensor.var.TensorVariable'>
TensorType(float64, scalar)
TensorType(float64, scalar)
True


## Logistic Regression

In [15]:
import theano
import theano.tensor as T

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [18]:
# Logisitic Function
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = theano.function([x], s)

# test
logistic([[0,1],[-1,-2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [19]:
# Alternate formulation
s2 = (1 + T.tanh(x / 2)) / 2
logistic2 = theano.function([x], s2)

# test
logistic2([[0,1],[-1,-2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [21]:
# Computing more than one thing at the same time
a, b = T.dmatrices('a','b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2
f = theano.function([a,b], [diff, abs_diff, diff_squared])

In [22]:
f([[1, 1], [1, 1]],[[0, 1], [2, 3]])

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]])]

In [23]:
# Setting a Default Value for an argument
from theano import Param
from theano import function
x, y = T.dscalars('x', 'y')
z = x + y
f = function([x, Param(y, default=1)], z)

print f(33)

print f(33,2)

34.0
35.0


In [25]:
x, y, w = T.dscalars('x','y','w')
z = (x + y) * w
f = function([x, Param(y, default=1), Param(w,default=2, name='w_by_name')], z)

print f(33)
print f(33, 2)
print f(33, 0, 1)
print f(33, w_by_name=1)
print f(33, w_by_name=1, y=0)

68.0
70.0
33.0
34.0
33.0


# Using shared variables

In [38]:
# Accumulator
from theano import shared
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])

print state.get_value()
print accumulator(1)
print state.get_value()
print accumulator(300)
print state.get_value()
print state.set_value(-1)
print accumulator(3)
print state.get_value()

0
0
1
1
301
None
-1
2


In [39]:
# Decrementor
decrementor = function([inc], state, updates=[(state, state-inc)])

print decrementor(2)
print state.get_value()

2
0


In [41]:
fn_of_state = state * 2 +inc

foo = T.scalar(dtype=state.dtype)
skip_shared = function([inc, foo], fn_of_state, givens=[(state, foo)])
skip_shared(1, 3)

array(7)

In [42]:
state.get_value()

array(0)

# Using Random Numbers

In [2]:
from theano.tensor.shared_randomstreams import RandomStreams
from theano import function
srng = RandomStreams(seed=234)
rv_u = srng.uniform((2,2))
rv_n = srng.normal((2,2))
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True)
nearly_zeros = function([], rv_u + rv_u - 2 *rv_u)

In [3]:
f_val0 = f()
f_val1 = f()

In [6]:
print f_val0, f_val1

[[ 0.12672381  0.97091597]
 [ 0.13989098  0.88754825]] [[ 0.31971415  0.47584377]
 [ 0.24129163  0.42046081]]


In [7]:
g_val0 = g()
g_val1 = g()

In [8]:
print g_val0, g_val1

[[ 0.37328447 -0.65746672]
 [-0.36302373 -0.97484625]] [[ 0.37328447 -0.65746672]
 [-0.36302373 -0.97484625]]


In [13]:
rv_u1 = srng.uniform((2,2))
rv_u2 = srng.uniform((2,2))
rv_u3 = srng.uniform((2,2))

nearly_zeros = function([], rv_u1 + rv_u2 - 2 *rv_u3)

nz_val0 = nearly_zeros()
print nz_val0

[[ 0.04680037 -1.11075443]
 [-1.19278792  0.33583672]]


In [16]:
rng_val = rv_u.rng.get_value(borrow=True)
rng_val.seed(89234)
rv_u.rng.set_value(rng_val, borrow=True)

In [17]:
srng.seed(902340)

In [20]:
state_after_v0 = rv_u.rng.get_value().get_state()
nearly_zeros()
v1 = f()
rng = rv_u.rng.get_value(borrow=True)
rng.set_state(state_after_v0)
rv_u.rng.set_value(rng, borrow=True)
v2 = f()
v3 = f()

## Copying Random State Between Theano Graphs

In [21]:
from __future__ import print_function
import theano
import numpy
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams
from theano.tensor.shared_randomstreams import RandomStreams

In [22]:
class Graph():
    def __init__(self, seed=123):
        self.rng = RandomStreams(seed)
        self.y = self.rng.uniform(size=(1,))

In [24]:
g1 = Graph(seed=123)
f1 = theano.function([], g1.y)

In [25]:
g2 = Graph(seed=987)
f2 = theano.function([], g2.y)

In [36]:
f1()

array([ 0.86608782])

In [38]:
f2()

array([ 0.3036794])

In [39]:
def copy_random_state(g1, g2):
    if isinstance(g1.rng, MRG_RandomStreams):
        g2.rng.rstate = g1.rng.rstate
    for (su1, su2) in zip(g1.rng.state_updates, g2.rng.state_updates):
        su2[0].set_value(su1[0].get_value())

In [40]:
copy_random_state(g1,g2)

In [41]:
f1()

array([ 0.35008632])

In [42]:
f2()

array([ 0.35008632])

## Logistic Regression

In [10]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

N = 400
feats = 784
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rng.randn(feats), name="w")
b = theano.shared(0., name="b")
print("Initial model:")
print(w.get_value())
print(b.get_value())

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))
prediction = p_1 > 0.5
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1)
cost = xent.mean() + 0.01 * (w ** 2).sum()
gw, gb = T.grad(cost, [w, b])

# Compile
train = theano.function(
    inputs=[x,y],
    outputs=[prediction, xent],
    updates=((w, w - 0.1 * gw), (b, b - 0.1 *gb))
)

predict = theano.function(inputs=[x], outputs=prediction)

Initial model:
[ -1.46964818e+00   6.96312035e-01   5.63136606e-01  -5.76561381e-02
  -1.30938311e-01  -6.34266843e-01   9.39973239e-01  -5.49881634e-01
   9.96338149e-01   5.91928243e-01  -6.32222587e-01  -1.33896320e+00
  -2.11788179e+00   1.37291963e+00  -3.03073846e-01  -2.59278089e-01
  -1.78440454e+00  -7.94201199e-01   1.86413197e-01  -2.14641774e+00
   2.46215465e-02  -2.68209227e-02  -2.06958784e+00   1.08268789e+00
  -1.16077520e-02   3.70991978e-01   3.69031830e-01  -8.37916056e-02
  -5.81819637e-01  -5.24289983e-02   4.90054345e-01   9.85959243e-01
   1.38232137e+00   1.36658208e+00  -1.49072529e-01  -1.34955463e+00
  -4.75950498e-01  -4.51070390e-01  -3.91773764e-01   8.63604756e-01
   1.21059034e+00  -2.76412836e+00   1.36001138e+00   1.03279773e+00
   1.16465091e+00   7.22919033e-01   4.25946568e-01   9.85843074e-01
  -1.29811892e+00  -5.43321568e-01  -1.93498551e+00  -9.70925941e-01
   6.26195464e-01   9.64514740e-01  -1.22660690e+00   4.78480143e-01
   3.98753736e-01  

In [11]:
# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])
    
print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Final model:
[  2.00263740e-03   5.37146174e-02   4.50001535e-02   1.52045934e-01
   7.88968215e-02  -1.08238408e-01  -9.19046220e-02   9.95753583e-02
  -9.00593466e-02  -2.21146729e-02  -2.44477203e-02  -1.78590049e-02
  -8.31543567e-02   1.66783638e-01   1.93382198e-01  -1.42203476e-01
   3.18997744e-02   1.05029807e-01  -9.82174681e-03   3.82740597e-02
   1.91028091e-02  -3.21284109e-03  -3.36250193e-02   1.16821798e-02
   1.73341070e-01  -1.75344352e-02   9.40477749e-02   8.08189786e-02
   7.41111725e-02   5.30399031e-02  -2.85683227e-02   3.43928487e-02
   1.76752267e-02  -5.88975473e-03  -1.04945177e-01   2.21947539e-01
  -3.06043145e-02   6.48267930e-02   6.42919505e-02  -2.91245706e-02
   1.49267662e-01   4.39646891e-02   9.45672761e-02   1.05620406e-01
  -7.09403130e-02   3.21625658e-02  -7.46947456e-02   9.59507320e-02
   9.53876401e-02   6.91097114e-02   8.82007084e-02   2.96600137e-02
   5.74718173e-02  -1.87618386e-02   9.80709394e-02   5.30611488e-02
   1.89940355e-03   7

In [2]:
theano.printing.pprint(prediction)

'gt((TensorConstant{1} / (TensorConstant{1} + exp(((-(x \\dot w)) - b)))), TensorConstant{0.5})'

In [3]:
theano.printing.debugprint(prediction)

Elemwise{gt,no_inplace} [@A] ''   
 |Elemwise{true_div,no_inplace} [@B] ''   
 | |DimShuffle{x} [@C] ''   
 | | |TensorConstant{1} [@D]
 | |Elemwise{add,no_inplace} [@E] ''   
 |   |DimShuffle{x} [@F] ''   
 |   | |TensorConstant{1} [@D]
 |   |Elemwise{exp,no_inplace} [@G] ''   
 |     |Elemwise{sub,no_inplace} [@H] ''   
 |       |Elemwise{neg,no_inplace} [@I] ''   
 |       | |dot [@J] ''   
 |       |   |x [@K]
 |       |   |w [@L]
 |       |DimShuffle{x} [@M] ''   
 |         |b [@N]
 |DimShuffle{x} [@O] ''   
   |TensorConstant{0.5} [@P]


In [4]:
theano.printing.debugprint(predict)

Elemwise{Composite{GT(scalar_sigmoid((-((-i0) - i1))), i2)}} [@A] ''   4
 |CGemv{inplace} [@B] ''   3
 | |Alloc [@C] ''   2
 | | |TensorConstant{0.0} [@D]
 | | |Shape_i{0} [@E] ''   1
 | |   |x [@F]
 | |TensorConstant{1.0} [@G]
 | |x [@F]
 | |w [@H]
 | |TensorConstant{0.0} [@D]
 |InplaceDimShuffle{x} [@I] ''   0
 | |b [@J]
 |TensorConstant{(1,) of 0.5} [@K]


In [9]:
theano.printing.pydotprint(prediction, outfile="logreg_pydotprint_prediction.eps", var_with_name_simple=True, format='eps') 

The output file is available at logreg_pydotprint_prediction.eps


In [14]:
import pydot
import graphviz

ImportError: No module named graphviz